# HOMEWORK 3 
# CS584 MACHINE LEARNING 
# JULEN FERRO BAÑALES
This Jupyter Notebook will include the coding of the Homework 3 related to the subject CS584 Machine Learning, as well as some comments made.

In [997]:
import numpy as np
import pandas as pd
import matplotlib as plt
import csv
import os
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)



Now, some basic auxiliary functions will be defined.

In [998]:

def cleaning(df):
    df = df.drop(['rightdown'], axis=1, level= 1)
    df = df.drop(['topleft'], axis=1, level= 1)
    return df

The first thing that must be done, is to read the CSV file with the location data that has been obtained after processing the mice images.

In [999]:
 
# reading the CSV file
df = pd.read_csv(r"C:\Users\ferro\Desktop\Disco Duro (17-08-2022) PRE-CHICAGO\DRIVE\University ( 28-11-2020 )\Master\Chicago IIT\Julen Subjects CHICAGO IIT\S1\CS 584 Machine Learning\Assignments\HW_3\A20512110_CollectedData_annotation.csv", header = [0, 1, 2, 3])
df = pd.DataFrame(df) 

size = os.path.getsize(r"C:\Users\ferro\Desktop\Disco Duro (17-08-2022) PRE-CHICAGO\DRIVE\University ( 28-11-2020 )\Master\Chicago IIT\Julen Subjects CHICAGO IIT\S1\CS 584 Machine Learning\Assignments\HW_3\A20512110_CollectedData_annotation.csv")

# displaying the contents of the CSV file
df.head()


scorer Unnamed: 1_level_0  \
    individuals Unnamed: 1_level_1   
      bodyparts Unnamed: 1_level_2   
         coords Unnamed: 1_level_3   
0  labeled-data               24.0   
1  labeled-data               24.0   
2  labeled-data               24.0   
3  labeled-data               24.0   
4  labeled-data               24.0   

                                  Unnamed: 2_level_0  annotation              \
                                  Unnamed: 2_level_1      mouse1               
                                  Unnamed: 2_level_2     topleft               
                                  Unnamed: 2_level_3           x           y   
0  A_male_in_a_new_cage_face_view_3_2022-08-10_15...  273.048052   12.903231   
1  A_male_in_a_new_cage_face_view_3_2022-08-10_15...  286.152150  140.449787   
2  A_male_in_a_new_cage_face_view_3_2022-08-10_15...  238.977396  154.427492   
3  A_male_in_a_new_cage_face_view_3_2022-08-10_15...  230.241331  166.657984   
4  A_male_in_a_new_cage_face_view_3_2022-08-10_15...  233.735757  173.646836   

                                                                           \
                                                                            
    rightdown                    nose                 leftear               
            x           y           x           y           x           y   
0  544.739689  331.769622  517.657886   21.639296  504.553788   51.341919   
1  516.784280  315.171098  515.910673  175.394049  488.828870  148.312246   
2  394.479362  225.189623  247.713462  212.085524  267.806412  203.349459   
3  310.613133  239.167328  287.899363  192.866180  273.048052  184.130115   
4  299.256248  261.007491  240.724609  187.624541  251.207888  191.118967   

                                                                           \
                                                                            
     rightear                 leftHip                rightHip               
            x           y           x           y           x           y   
0  464.367887   59.204378  532.509198  169.278803  481.840018  168.405197   
1  457.379034  152.680279  478.345591  234.799295  425.055592  229.557655   
2  272.174445  198.107820  300.129855  198.107820  303.624281  187.624541   
3  280.036904  196.360606  253.828707  196.360606  269.553625  210.338311   
4  258.196740  185.003721  257.323134  208.591098  287.899363  194.613393   

                                                                             \
                                                   mouse2                     
     tailBase                 tailEnd             topleft     rightdown       
            x           y           x           y       x   y         x   y   
0  462.620673  251.397819  307.118707  295.951753     NaN NaN       NaN NaN   
1  412.825100  262.754704  293.141002  285.468475     NaN NaN       NaN NaN   
2  336.821330  189.371754  404.089034  217.327164     NaN NaN       NaN NaN   
3  245.092642  221.695196  303.624281  234.799295     NaN NaN       NaN NaN   
4  281.784117  224.316016  252.955101  259.260278     NaN NaN       NaN NaN   

                                                                           \
                                                                            
  nose     leftear     rightear     leftHip     rightHip     tailBase       
     x   y       x   y        x   y       x   y        x   y        x   y   
0  NaN NaN     NaN NaN      NaN NaN     NaN NaN      NaN NaN      NaN NaN   
1  NaN NaN     NaN NaN      NaN NaN     NaN NaN      NaN NaN      NaN NaN   
2  NaN NaN     NaN NaN      NaN NaN     NaN NaN      NaN NaN      NaN NaN   
3  NaN NaN     NaN NaN      NaN NaN     NaN NaN      NaN NaN      NaN NaN   
4  NaN NaN     NaN NaN      NaN NaN     NaN NaN      NaN NaN      NaN NaN   

               
               
  tailEnd      
        x   y  
0     NaN NaN  
1     NaN NaN  
2     NaN NaN  
3     NaN NaN  
4     NaN NaN

 Now, the dataset will be processed in order to get the dataset structure needed to feed the Neural Network. 

In [1000]:
rows = len(df) #getting the number of data examples that we have

# if we want to sample the data in order to mix it
df = df.sample(frac = 1.00)

#getting rid of the possible NaN values
th = 1
df.dropna(axis = 0, inplace = True)

rows_na = len(df)

print("\n")
print("{} rows were eliminated due to issues related to NaN values".format(rows - rows_na))
print("\n")


df = df.iloc[: , 3:-1]
mouse1 = df.xs('mouse1', level=1, axis=1)
mouse2 = df.xs('mouse2', level=1, axis=1)

# we add as rows the mouse2 samples' data taken in pictures where more than one mice were found
df = pd.concat([mouse1, mouse2], axis=0)

# we create one dataset per axis with data related to its own axis
df_x = df.xs('x', level=2, axis=1)
df_y = df.xs('y', level=2, axis=1)

# we calculate the average location of the mice in both axis
x_cg = (df_x.xs('topleft', level=1, axis=1) + df_x.xs('rightdown', level=1, axis=1)) / 2
y_cg = (df_y.xs('topleft', level=1, axis=1) + df_y.xs('rightdown', level=1, axis=1)) / 2

# renaming the columns of the dataframes
x_cg.rename(columns = {'annotation':'x_location'}, inplace = True)
y_cg.rename(columns = {'annotation':'y_location'}, inplace = True)

# we get rid of the data used for calculating the average location of the mice in both axis
df = cleaning(df)
df_x = cleaning(df_x)
df_y = cleaning(df_y)

# we add to df the calculated average location of the mice in both axis
df = df.assign(x_locatin = x_cg)
df = df.assign(y_location = y_cg)

#print(df_x)
# df_x is a dataset made up of 187 rows and 7 columns corresponding to the attributes

#print(df_y)
# df_y is a dataset made up of 187 rows and 7 columns corresponding to the attributes

print(df)
# df is a dataset made up of 187 rows and 16 columns ( 7 attributes per axis + one location per axis)

#print(df.shape)



64 rows were eliminated due to issues related to NaN values


     annotation                                                              \
        leftHip                 leftear                    nose               
              x           y           x           y           x           y   
31   294.888215  212.959131  302.750674  204.223065  320.222805  199.855033   
124  235.482970  185.877328  198.791495  163.163557  171.709692  175.394049   
86   214.516413  132.587328  191.802642   69.687656  202.285921   45.226673   
121  394.479362  252.271426  469.609526  246.156180  493.196903  248.777000   
123  132.397397  166.657984   62.508873   98.516673   73.865758   88.033394   
110  128.902971  189.371754  149.869528  114.241591  145.501495   97.643066   
26   258.196740  218.200770  250.334281  214.706344  227.620511  239.167328   
106  143.754282  236.546508  174.330511  206.843885  198.791495  235.672901   
45   111.430839  156.174705   93.958708   74.055689   56.393627   6

In this next step, the fed datasets will be divided in training and testing datasets.

In [1001]:
coef = 0.2

# x_cg and y_cg will be the respective results for the testing and training of the 7 atributes fed Neural Network
# y_2d will be the respective result for the testing and training of the 14 atributes fed Neural Network

y_2d = pd.concat([x_cg, y_cg], axis = 1)

in_x_train, in_x_test, out_x_train, out_x_test = train_test_split(df_x, x_cg, test_size = coef)
in_y_train, in_y_test, out_y_train,  out_y_test = train_test_split(df_y, y_cg, test_size = coef)
in_2d_train, in_2d_test, out_2d_train, out_2d_test = train_test_split(df, y_2d, test_size = coef)

# just checking if the split is done properly according to the established 0.2 coefficient 

# print(in_x_test.shape)
# print(in_y_train.shape)
# print(in_2d_train.shape)
# print(in_2d_test.shape)




Here, the first model will be ran and its name will be "THE DEPENDENT MODEL". This model is caracterized by taking the 14 attributes as input, and giving two outputs as both coordinates of the location of the mouse.

As the first step, the Neural Network model will be created by using the Tensorflow library.

In [1002]:
# For "THE DEPENDENT MODEL" the next datasets will be used: in_2d_train, in_2d_test, out_2d_train, out_2d_test

import tensorflow as tf

input_shape = [in_2d_train.shape[1]]

model = tf.keras.Sequential([
    tf.keras.layers.Dense(units=64, activation='relu', input_shape=input_shape),
    tf.keras.layers.Dense(units=64, activation='relu'),
    tf.keras.layers.Dense(units=2)
    ])
 
# printing a summary in order to check if it works
model.summary()

Model: "sequential_39"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_115 (Dense)           (None, 64)                1088      
                                                                 
 dense_116 (Dense)           (None, 64)                4160      
                                                                 
 dense_117 (Dense)           (None, 2)                 130       
                                                                 
Total params: 5,378
Trainable params: 5,378
Non-trainable params: 0
_________________________________________________________________


Now, we have to keep building up the Neural Network by establishing some other parameters and then the model will be ready for training. 

In [1003]:
# We will use the adam optimizer due to the fact that it behaves well overally 

model.compile(optimizer='adam', loss='mae')

batchs = int(rows / 30)
print(batchs)
losses = model.fit(
                    in_2d_train, out_2d_train,
                    batch_size = batchs,
                    epochs = 15 
                   )

model.summary()

4
Epoch 1/15
25/25 [==============================] - 1s 3ms/step - loss: nan
Epoch 2/15
25/25 [==============================] - 0s 2ms/step - loss: nan
Epoch 3/15
25/25 [==============================] - 0s 1ms/step - loss: nan
Epoch 4/15
25/25 [==============================] - 0s 2ms/step - loss: nan
Epoch 5/15
25/25 [==============================] - 0s 2ms/step - loss: nan
Epoch 6/15
25/25 [==============================] - 0s 2ms/step - loss: nan
Epoch 7/15
25/25 [==============================] - 0s 2ms/step - loss: nan
Epoch 8/15
25/25 [==============================] - 0s 2ms/step - loss: nan
Epoch 9/15
25/25 [==============================] - 0s 2ms/step - loss: nan
Epoch 10/15
25/25 [==============================] - 0s 2ms/step - loss: nan
Epoch 11/15
25/25 [==============================] - 0s 2ms/step - loss: nan
Epoch 12/15
25/25 [==============================] - 0s 2ms/step - loss: nan
Epoch 13/15
25/25 [==============================] - 0s 1ms/step - loss: nan
Epoch 